In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import warnings

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'lifeexpectancy'
table_name = 'lifeexpectancy'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

lifeexpectancy_df = pd.read_sql_query('select * from lifeexpectancy',con=engine)

# no need for an open connection, 
# as we're only doing a single query
engine.dispose()

In [2]:
# What does the data look like?
lifeexpectancy_df.head()

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
0,Afghanistan,2015,Developing,65.0,263.0,62,0.01,71.279624,65.0,1154,...,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1
1,Afghanistan,2014,Developing,59.9,271.0,64,0.01,73.523582,62.0,492,...,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0
2,Afghanistan,2013,Developing,59.9,268.0,66,0.01,73.219243,64.0,430,...,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9
3,Afghanistan,2012,Developing,59.5,272.0,69,0.01,78.184215,67.0,2787,...,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8
4,Afghanistan,2011,Developing,59.2,275.0,71,0.01,7.097109,68.0,3013,...,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5


In [3]:
# What types of data are in the df?
lifeexpectancy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
Country                            2938 non-null object
Year                               2938 non-null int64
Status                             2938 non-null object
Life expectancy                    2928 non-null float64
Adult Mortality                    2928 non-null float64
infant deaths                      2938 non-null int64
Alcohol                            2744 non-null float64
percentage expenditure             2938 non-null float64
Hepatitis B                        2385 non-null float64
Measles                            2938 non-null int64
 BMI                               2904 non-null float64
under-five deaths                  2938 non-null int64
Polio                              2919 non-null float64
Total expenditure                  2712 non-null float64
Diphtheria                         2919 non-null float64
 HIV/AIDS                          2938 non-null

In [4]:
# how many nulls in the set?
lifeexpectancy_df.isnull().sum()

Country                              0
Year                                 0
Status                               0
Life expectancy                     10
Adult Mortality                     10
infant deaths                        0
Alcohol                            194
percentage expenditure               0
Hepatitis B                        553
Measles                              0
 BMI                                34
under-five deaths                    0
Polio                               19
Total expenditure                  226
Diphtheria                          19
 HIV/AIDS                            0
GDP                                448
Population                         652
 thinness  1-19 years               34
 thinness 5-9 years                 34
Income composition of resources    167
Schooling                          163
dtype: int64

In [5]:
# separate the columns that have nulls for imputation
cols_with_null = []
for col in lifeexpectancy_df.columns:
    if lifeexpectancy_df[col].isnull().sum() > 0:
        cols_with_null.append(col)
print(cols_with_null)

['Life expectancy ', 'Adult Mortality', 'Alcohol', 'Hepatitis B', ' BMI ', 'Polio', 'Total expenditure', 'Diphtheria ', 'GDP', 'Population', ' thinness  1-19 years', ' thinness 5-9 years', 'Income composition of resources', 'Schooling']


In [6]:
# make a copy in case something bad happens!
le2 = lifeexpectancy_df.copy()

# imputation will be done by country
countries = lifeexpectancy_df["Country"].unique()

# imputation using each country's mean value
for col in cols_with_null:
    for c in countries:
        le2.loc[le2["Country"] == c, col].fillna(le2[le2["Country"] == c][col].mean())

In [7]:
# how much effect did imputation have?
for col in cols_with_null:
    print("Statistics for columns: {}".format(col))
    print(pd.concat([lifeexpectancy_df[col], le2[col]], axis=1).describe())

Statistics for columns: Life expectancy 
       Life expectancy   Life expectancy 
count       2928.000000       2928.000000
mean          69.224932         69.224932
std            9.523867          9.523867
min           36.300000         36.300000
25%           63.100000         63.100000
50%           72.100000         72.100000
75%           75.700000         75.700000
max           89.000000         89.000000
Statistics for columns: Adult Mortality
       Adult Mortality  Adult Mortality
count      2928.000000      2928.000000
mean        164.796448       164.796448
std         124.292079       124.292079
min           1.000000         1.000000
25%          74.000000        74.000000
50%         144.000000       144.000000
75%         228.000000       228.000000
max         723.000000       723.000000
Statistics for columns: Alcohol
           Alcohol      Alcohol
count  2744.000000  2744.000000
mean      4.602861     4.602861
std       4.052413     4.052413
min       0.010000   

Based on the output of the above cell, imputation failed...no nulls have been filled in

In [8]:
# trying to identify where the problems are
le3 = lifeexpectancy_df.copy()
for col in cols_with_null:
    for country in countries:
        missing_number = le3.loc[le3["Country"] == country, col].isnull().sum()
        if missing_number>0:
            print("total missing ratio: ", missing_number/le3.loc[le3["Country"] == country, col].shape[0])
            print("replacement value: ", le3[le3["Country"] == country][col].mean())

total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan


total missing ratio:  0.0625
replacement value:  3.1546666666666665
total missing ratio:  0.0625
replacement value:  10.754666666666667
total missing ratio:  0.0625
replacement value:  0.8006666666666665
total missing ratio:  0.0625
replacement value:  0.010000000000000002
total missing ratio:  0.0625
replacement value:  7.588666666666667
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  0.0625
replacement value:  10.024000000000001
total missing ratio:  0.0625
replacement value:  2.096
total missing ratio:  0.0625
replacement value:  1.4579999999999997
total missing ratio:  0.0625
replacement value:  4.983999999999999
total missing ratio:  0.0625
replacement value:  4.463333333333334
total missing ratio:  0.0625
replacement value:  6.926666666666667
total missing ratio:  0.0625
replacement value:  10.338
total missing ratio:  0.0625
replacement value:  0.8046666666666666
total missing ratio:  0.0625
replacement value:  0.33066666666666666
total missing ratio:  0.

total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  0.375
replacement value:  84.3
total missing ratio:  0.6875
replacement value:  44.0
total missing ratio:  0.125
replacement value:  63.214285714285715
total missing ratio:  0.0625
replacement value:  4.604
total missing ratio:  0.0625
replacement value:  5.723333333333333
total missing ratio:  0.0625
replacement value:  3.919333333333333
total missing ratio:  0.0625
replacement value:  4.791333333333332
total missing ratio:  0.0625
replacement value:  6.9126666666666665
total missing ratio:  0.0625
replacement value:  4.8613333333333335
total missing ratio:  0.0625
replacement value:  8.836666666666668
total missing ratio:  0.0625
replacement value:  4.715333333333333
total missing ratio:  0.0625
replacement value:  5.679333333333333
total missing ratio:  0.0625
replacement value:  6.706
total missing ratio:  0.0625

replacement value:  7.7219999999999995
total missing ratio:  0.0625
replacement value:  5.462666666666666
total missing ratio:  0.0625
replacement value:  6.778666666666667
total missing ratio:  0.0625
replacement value:  4.926666666666667
total missing ratio:  0.0625
replacement value:  3.905333333333333
total missing ratio:  0.0625
replacement value:  6.379333333333333
total missing ratio:  0.0625
replacement value:  7.952666666666667
total missing ratio:  0.0625
replacement value:  2.6013333333333337
total missing ratio:  0.0625
replacement value:  5.912
total missing ratio:  0.0625
replacement value:  6.956666666666666
total missing ratio:  0.0625
replacement value:  5.2780000000000005
total missing ratio:  0.0625
replacement value:  6.223333333333333
total missing ratio:  0.0625
replacement value:  6.350666666666666
total missing ratio:  0.0625
replacement value:  6.922666666666667
total missing ratio:  0.0625
replacement value:  4.816
total missing ratio:  0.0625
replacement valu

total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan
total missing ratio:  1.0
replacement value:  nan


In [9]:
# find the problematic country/column pairs
le3 = lifeexpectancy_df.copy()
for col in cols_with_null:
    for country in countries:
        missing_number = le3.loc[le3["Country"] == country, col].isnull().sum()
        if missing_number>0:
            print("column name: {}, country name: {}".format(col, country))
            print("total missing ratio: ", missing_number/le3.loc[le3["Country"] == country, col].shape[0])
            print("replacement value: ", le3[le3["Country"] == country][col].mean())

column name: Life expectancy , country name: Cook Islands
total missing ratio:  1.0
replacement value:  nan
column name: Life expectancy , country name: Dominica
total missing ratio:  1.0
replacement value:  nan
column name: Life expectancy , country name: Marshall Islands
total missing ratio:  1.0
replacement value:  nan
column name: Life expectancy , country name: Monaco
total missing ratio:  1.0
replacement value:  nan
column name: Life expectancy , country name: Nauru
total missing ratio:  1.0
replacement value:  nan
column name: Life expectancy , country name: Niue
total missing ratio:  1.0
replacement value:  nan
column name: Life expectancy , country name: Palau
total missing ratio:  1.0
replacement value:  nan
column name: Life expectancy , country name: Saint Kitts and Nevis
total missing ratio:  1.0
replacement value:  nan
column name: Life expectancy , country name: San Marino
total missing ratio:  1.0
replacement value:  nan
column name: Life expectancy , country name: Tuva

column name: Alcohol, country name: Indonesia
total missing ratio:  0.0625
replacement value:  0.06866666666666667
column name: Alcohol, country name: Iran (Islamic Republic of)
total missing ratio:  0.0625
replacement value:  0.016
column name: Alcohol, country name: Iraq
total missing ratio:  0.0625
replacement value:  0.13466666666666666
column name: Alcohol, country name: Ireland
total missing ratio:  0.0625
replacement value:  12.654
column name: Alcohol, country name: Israel
total missing ratio:  0.0625
replacement value:  2.497333333333333
column name: Alcohol, country name: Italy
total missing ratio:  0.0625
replacement value:  8.266666666666667
column name: Alcohol, country name: Jamaica
total missing ratio:  0.0625
replacement value:  3.624
column name: Alcohol, country name: Japan
total missing ratio:  0.0625
replacement value:  7.040666666666667
column name: Alcohol, country name: Jordan
total missing ratio:  0.0625
replacement value:  0.5173333333333333
column name: Alcoho

total missing ratio:  0.0625
replacement value:  1.1673333333333331
column name: Alcohol, country name: Trinidad and Tobago
total missing ratio:  0.0625
replacement value:  6.044666666666667
column name: Alcohol, country name: Tunisia
total missing ratio:  0.0625
replacement value:  1.3106666666666666
column name: Alcohol, country name: Turkey
total missing ratio:  0.0625
replacement value:  1.4213333333333333
column name: Alcohol, country name: Turkmenistan
total missing ratio:  0.0625
replacement value:  2.6546666666666665
column name: Alcohol, country name: Uganda
total missing ratio:  0.0625
replacement value:  8.050666666666666
column name: Alcohol, country name: Ukraine
total missing ratio:  0.0625
replacement value:  7.369333333333333
column name: Alcohol, country name: United Arab Emirates
total missing ratio:  0.0625
replacement value:  1.75
column name: Alcohol, country name: United Republic of Tanzania
total missing ratio:  0.0625
replacement value:  3.582666666666667
column

column name: Hepatitis B, country name: Montenegro
total missing ratio:  0.375
replacement value:  48.5
column name: Hepatitis B, country name: Mozambique
total missing ratio:  0.0625
replacement value:  67.86666666666666
column name: Hepatitis B, country name: Myanmar
total missing ratio:  0.1875
replacement value:  65.46153846153847
column name: Hepatitis B, country name: Namibia
total missing ratio:  0.625
replacement value:  86.33333333333333
column name: Hepatitis B, country name: Nepal
total missing ratio:  0.1875
replacement value:  65.38461538461539
column name: Hepatitis B, country name: Netherlands
total missing ratio:  0.6875
replacement value:  48.2
column name: Hepatitis B, country name: Niger
total missing ratio:  0.5625
replacement value:  60.57142857142857
column name: Hepatitis B, country name: Nigeria
total missing ratio:  0.3125
replacement value:  42.90909090909091
column name: Hepatitis B, country name: Norway
total missing ratio:  1.0
replacement value:  nan
colum

replacement value:  2.929333333333333
column name: Total expenditure, country name: Eritrea
total missing ratio:  0.0625
replacement value:  3.4686666666666666
column name: Total expenditure, country name: Estonia
total missing ratio:  0.0625
replacement value:  5.687333333333333
column name: Total expenditure, country name: Ethiopia
total missing ratio:  0.0625
replacement value:  4.901333333333334
column name: Total expenditure, country name: Fiji
total missing ratio:  0.0625
replacement value:  3.8973333333333335
column name: Total expenditure, country name: Finland
total missing ratio:  0.0625
replacement value:  8.614
column name: Total expenditure, country name: France
total missing ratio:  0.0625
replacement value:  6.556
column name: Total expenditure, country name: Gabon
total missing ratio:  0.0625
replacement value:  3.175333333333333
column name: Total expenditure, country name: Gambia
total missing ratio:  0.0625
replacement value:  5.308666666666666
column name: Total exp

total missing ratio:  0.0625
replacement value:  7.7219999999999995
column name: Total expenditure, country name: Papua New Guinea
total missing ratio:  0.0625
replacement value:  5.462666666666666
column name: Total expenditure, country name: Paraguay
total missing ratio:  0.0625
replacement value:  6.778666666666667
column name: Total expenditure, country name: Peru
total missing ratio:  0.0625
replacement value:  4.926666666666667
column name: Total expenditure, country name: Philippines
total missing ratio:  0.0625
replacement value:  3.905333333333333
column name: Total expenditure, country name: Poland
total missing ratio:  0.0625
replacement value:  6.379333333333333
column name: Total expenditure, country name: Portugal
total missing ratio:  0.0625
replacement value:  7.952666666666667
column name: Total expenditure, country name: Qatar
total missing ratio:  0.0625
replacement value:  2.6013333333333337
column name: Total expenditure, country name: Republic of Korea
total missi

column name: GDP, country name: Micronesia (Federated States of)
total missing ratio:  1.0
replacement value:  nan
column name: GDP, country name: Monaco
total missing ratio:  1.0
replacement value:  nan
column name: GDP, country name: Niue
total missing ratio:  1.0
replacement value:  nan
column name: GDP, country name: Papua New Guinea
total missing ratio:  0.0625
replacement value:  758.0976163466668
column name: GDP, country name: Republic of Korea
total missing ratio:  1.0
replacement value:  nan
column name: GDP, country name: Republic of Moldova
total missing ratio:  1.0
replacement value:  nan
column name: GDP, country name: Saint Kitts and Nevis
total missing ratio:  1.0
replacement value:  nan
column name: GDP, country name: Saint Lucia
total missing ratio:  1.0
replacement value:  nan
column name: GDP, country name: Saint Vincent and the Grenadines
total missing ratio:  1.0
replacement value:  nan
column name: GDP, country name: San Marino
total missing ratio:  1.0
replaceme

column name: Income composition of resources, country name: Republic of Korea
total missing ratio:  1.0
replacement value:  nan
column name: Income composition of resources, country name: Republic of Moldova
total missing ratio:  1.0
replacement value:  nan
column name: Income composition of resources, country name: San Marino
total missing ratio:  1.0
replacement value:  nan
column name: Income composition of resources, country name: Somalia
total missing ratio:  1.0
replacement value:  nan
column name: Income composition of resources, country name: Tuvalu
total missing ratio:  1.0
replacement value:  nan
column name: Income composition of resources, country name: United Kingdom of Great Britain and Northern Ireland
total missing ratio:  1.0
replacement value:  nan
column name: Income composition of resources, country name: United Republic of Tanzania
total missing ratio:  1.0
replacement value:  nan
column name: Income composition of resources, country name: United States of America


In [10]:
# trying to break down the imputation into a single example to see if there was a mistake in the code...
le2.loc[le2["Country"] == 'Sierra Leone']

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
2298,Sierra Leone,2015,Developing,51.0,413.0,22,NaN,0.000000,86.0,607,...,86.0,NaN,86.0,0.5,587.538233,723725.0,7.4,7.3,0.431,9.5
2299,Sierra Leone,2014,Developing,48.1,463.0,23,0.01,1.443286,83.0,1006,...,83.0,11.90,83.0,0.6,78.439476,779162.0,7.5,7.4,0.426,9.5
2300,Sierra Leone,2013,Developing,54.0,47.0,23,0.01,1.321464,92.0,15,...,92.0,11.59,92.0,0.8,71.818700,692279.0,7.7,7.6,0.413,9.3
2301,Sierra Leone,2012,Developing,49.7,411.0,25,0.01,54.560337,91.0,678,...,91.0,11.24,91.0,0.9,561.898424,676613.0,7.9,7.8,0.401,9.1
2302,Sierra Leone,2011,Developing,48.9,418.0,26,3.78,54.665917,89.0,1865,...,88.0,11.98,89.0,1.3,445.525000,6611692.0,8.1,8.0,0.392,8.9
2303,Sierra Leone,2010,Developing,48.1,424.0,27,3.84,5.347718,86.0,1089,...,84.0,1.32,86.0,1.6,45.128418,645872.0,8.3,8.2,0.384,8.7
2304,Sierra Leone,2009,Developing,47.1,433.0,28,3.97,49.837127,84.0,31,...,81.0,13.13,84.0,1.7,394.593244,63126.0,8.5,8.4,0.375,8.5
2305,Sierra Leone,2008,Developing,46.2,441.0,29,3.91,5.379606,77.0,44,...,75.0,1.29,77.0,1.9,46.375918,6165372.0,8.7,8.7,0.367,8.3
2306,Sierra Leone,2007,Developing,45.3,45.0,29,3.86,45.571089,63.0,0,...,63.0,1.12,64.0,2.2,358.827472,615417.0,8.9,8.9,0.357,8.2
2307,Sierra Leone,2006,Developing,44.3,464.0,30,3.80,38.000758,NaN,33,...,65.0,1.68,64.0,2.2,322.313468,5848692.0,9.1,9.1,0.348,8.0


In [11]:
le2.loc[le2["Country"] == 'Sierra Leone'].isnull().sum()

Country                            0
Year                               0
Status                             0
Life expectancy                    0
Adult Mortality                    0
infant deaths                      0
Alcohol                            1
percentage expenditure             0
Hepatitis B                        7
Measles                            0
 BMI                               0
under-five deaths                  0
Polio                              0
Total expenditure                  1
Diphtheria                         0
 HIV/AIDS                          0
GDP                                0
Population                         0
 thinness  1-19 years              0
 thinness 5-9 years                0
Income composition of resources    0
Schooling                          0
dtype: int64

In [12]:
# Sierra Leone has 7 nulls in Hepatitis B column, try to fill these in...
le2.loc[le2["Country"] == 'Sierra Leone', 'Hepatitis B'].fillna(le2[le2["Country"] == 'Sierra Leone']['Hepatitis B'].mean(), inplace=True)

In [13]:
le2.loc[le2["Country"] == 'Sierra Leone'].isnull().sum()

Country                            0
Year                               0
Status                             0
Life expectancy                    0
Adult Mortality                    0
infant deaths                      0
Alcohol                            1
percentage expenditure             0
Hepatitis B                        7
Measles                            0
 BMI                               0
under-five deaths                  0
Polio                              0
Total expenditure                  1
Diphtheria                         0
 HIV/AIDS                          0
GDP                                0
Population                         0
 thinness  1-19 years              0
 thinness 5-9 years                0
Income composition of resources    0
Schooling                          0
dtype: int64

In [14]:
# still 7 nulls, fillna did NOT work - check to make sure the portion inside fillna is working:
le2[le2["Country"] == 'Sierra Leone']['Hepatitis B'].mean()

83.44444444444444

In [15]:
# the portion inside fillna seems to be working, so why isn't it actually filling the nulls?
# try reassignment
le2.loc[le2["Country"] == 'Sierra Leone', 'Hepatitis B'] = le2.loc[le2["Country"] == 'Sierra Leone', 'Hepatitis B'].fillna(le2[le2["Country"] == 'Sierra Leone']['Hepatitis B'].mean())

In [16]:
le2.loc[le2["Country"] == 'Sierra Leone'].isnull().sum()

Country                            0
Year                               0
Status                             0
Life expectancy                    0
Adult Mortality                    0
infant deaths                      0
Alcohol                            1
percentage expenditure             0
Hepatitis B                        0
Measles                            0
 BMI                               0
under-five deaths                  0
Polio                              0
Total expenditure                  1
Diphtheria                         0
 HIV/AIDS                          0
GDP                                0
Population                         0
 thinness  1-19 years              0
 thinness 5-9 years                0
Income composition of resources    0
Schooling                          0
dtype: int64

In [17]:
# the 7 nulls were filled this time! confirm by looking at Sierra Leone's data again:
le2.loc[le2["Country"] == 'Sierra Leone']

,Country,Year,Status,Life expectancy,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,...,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling
2298,Sierra Leone,2015,Developing,51.0,413.0,22,NaN,0.000000,86.000000,607,...,86.0,NaN,86.0,0.5,587.538233,723725.0,7.4,7.3,0.431,9.5
2299,Sierra Leone,2014,Developing,48.1,463.0,23,0.01,1.443286,83.000000,1006,...,83.0,11.90,83.0,0.6,78.439476,779162.0,7.5,7.4,0.426,9.5
2300,Sierra Leone,2013,Developing,54.0,47.0,23,0.01,1.321464,92.000000,15,...,92.0,11.59,92.0,0.8,71.818700,692279.0,7.7,7.6,0.413,9.3
2301,Sierra Leone,2012,Developing,49.7,411.0,25,0.01,54.560337,91.000000,678,...,91.0,11.24,91.0,0.9,561.898424,676613.0,7.9,7.8,0.401,9.1
2302,Sierra Leone,2011,Developing,48.9,418.0,26,3.78,54.665917,89.000000,1865,...,88.0,11.98,89.0,1.3,445.525000,6611692.0,8.1,8.0,0.392,8.9
2303,Sierra Leone,2010,Developing,48.1,424.0,27,3.84,5.347718,86.000000,1089,...,84.0,1.32,86.0,1.6,45.128418,645872.0,8.3,8.2,0.384,8.7
2304,Sierra Leone,2009,Developing,47.1,433.0,28,3.97,49.837127,84.000000,31,...,81.0,13.13,84.0,1.7,394.593244,63126.0,8.5,8.4,0.375,8.5
2305,Sierra Leone,2008,Developing,46.2,441.0,29,3.91,5.379606,77.000000,44,...,75.0,1.29,77.0,1.9,46.375918,6165372.0,8.7,8.7,0.367,8.3
2306,Sierra Leone,2007,Developing,45.3,45.0,29,3.86,45.571089,63.000000,0,...,63.0,1.12,64.0,2.2,358.827472,615417.0,8.9,8.9,0.357,8.2
2307,Sierra Leone,2006,Developing,44.3,464.0,30,3.80,38.000758,83.444444,33,...,65.0,1.68,64.0,2.2,322.313468,5848692.0,9.1,9.1,0.348,8.0
